In [1]:
import numpy as np
import dxchange
import tomopy
import tomocupy

from ipywidgets import interact
import ipywidgets as widgets
import matplotlib.pyplot as plt
%matplotlib inline
import tkinter as tk
from tkinter.filedialog import askopenfile

/home/beams/TOMO/conda/miniforge3/envs/tomopyui/lib/python3.9/site-packages/cupyx/jit/_interface.py:173: FutureWarning: cupyx.jit.rawkernel is experimental. The interface can change in the future.
  cupy._util.experimental('cupyx.jit.rawkernel')


In [2]:
# Utility Functions

In [3]:
def read_filename():
    window = tk.Tk()
    window.wm_attributes('-topmost', 1)
    window.withdraw()

    file = askopenfile(mode ='r', filetypes =[('Python Files', '*.h5')])
    window.destroy()
    return file.name

# Load data

In [4]:
# !wget -nc https://anl.box.com/shared/static/or8vlzdu07d8zwxvk50ihwghq39ide0o.npz
# file_name = read_filename()
fname = '/data/2022-12/Luxi_173.h5'
# dark_file_name = file_name
# flat_file_name = file_name

# Read data, dark and flat fields, projection angles, metadata

In [5]:
# data = np.random.rand(100,256,256) #3d-array with 100 frames 256x256
data, flat, dark, theta = dxchange.read_aps_tomoscan_hdf5(fname)#, sino=(100, 400))
tt, meta_dict = dxchange.read_hdf_meta(fname)
# print(meta_dict)

In [6]:
data.shape 

(1800, 852, 2800)

# Plot projections

In [7]:
def plot_projections(angle=1):
    plt.imshow(data[angle,:],cmap='gray')
interact(plot_projections, angle = widgets.IntSlider(name='Projection', value=data.shape[0]//2,
                                               min=0,
                                               max=data.shape[0]-1,
                                               step=1))

interactive(children=(IntSlider(value=900, description='angle', max=1799), Output()), _dom_classes=('widget-in…

<function __main__.plot_projections(angle=1)>

In [11]:
from types import SimpleNamespace

JUPYTER_PARAMS = ('file-reading', 'remove-stripe',
                'reconstruction', 'fw', 'ti', 'vo-all', 'reconstruction-types', 'lamino', 'rotate-proj')
params_dict = {}
# for section in tomocupy.config.RECON_PARAMS:
for section in JUPYTER_PARAMS:
    # print(section)
    for key, value in tomocupy.config.SECTIONS[section].items():
        # print('%s = %s ' % (key, value['default']))
        key = key.replace('-', '_')
        params_dict[key] = value['default']

# print(params_dict)
args = SimpleNamespace(
                       file_name                    = fname,
                       dark_file_name               = fname, 
                       flat_file_name               = fname, 
                       out_path_name                = '/data/tmpfdc/',
                       file_type                    = 'standard',
                       binning                      = 0,
                       blocked_views                = 'none',
                       remove_stripe_method         = 'vo',
                       rotation_axis                = -1,
                       center_search_width          =  50,
                       center_search_step           =  0.5,
                       nsino                        = '0.5',
                       nsino_per_chunk              = 8,
                       nproj_per_chunk              = 8,
                       start_row                    = 0,
                       end_row                      = -1,
                       start_column                 = 0,
                       end_column                   = -1,
                       start_proj                   = 0,
                       end_proj                     = -1,
                       rotation_axis_auto           = 'auto', 
                       rotation_axis_pairs          = '[0,0]',
                       rotation_axis_sift_threshold = 0.5,
                       rotation_axis_method         = 'sift',  

                       find_center_start_row        = 0,
                       find_center_end_row          = -1,
                       dtype                        = 'float16',

                       clear_folder                 = True,
                       fbp_filter                   = 'shepp',
                       dezinger                     = 0,
                       max_write_threads            = 8,
                       max_read_threads             = 4,
                       minus_log                    = True,
                       flat_linear                  = 'False',
    
    
    
    
    
    
    
    
    
    
    
                       reconstruction_type          = 'try',
                       reconstruction_algorithm     = 'fourierrec',
                       lamino_search_width          =  5.0,
                       lamino_search_step           =  0.25,
                       lamino_angle                 =  0,
                       lamino_start_row             =  0,
                       lamino_end_row               =  -1,
                       rotate_proj_angle            = 0,
                       rotate_proj_order            = 1,
                       retrieve_phase_method        = None,
                       pixel_size                   = meta_dict['measurement_instrument_detection_system_objective_resolution'][0] * 1e-4,
                       propagation_distance         = meta_dict['measurement_instrument_detector_motor_stack_setup_z'][0],
                       energy                       = meta_dict['measurement_instrument_monochromator_energy'][0],
                       retrieve_phase_alpha         = 0.2,
                      )

args1 = SimpleNamespace(**params_dict)

args1.reconstruction_type    = 'try'
args1.file_name              = fname
# args1.dark_file_name         = fname, 
# args1.flat_file_name         = fname, 
args1.rotation_axis_auto     = 'auto'
args1.rotation_axis_method   = 'sift' 
args1.dtype                  = 'float16'
args1.out_path_name          = '/data/tmpfdc' 
args1.clear_folder           = True
args1.fbp_filter             = 'shepp' 
args1.retrieve_phase_method  = None 
args1.remove_stripe_method   = 'vo'
args1.pixel_size             = meta_dict['measurement_instrument_detection_system_objective_resolution'][0] * 1e-4
args1.propagation_distance   = meta_dict['measurement_instrument_detector_motor_stack_setup_z'][0]
args1.energy                 = meta_dict['measurement_instrument_monochromator_energy'][0]
args1.retrieve_phase_alpha   = 0.0008
args1.rotation_axis_sift_threshold = 0.5
args1.rotation_axis = -1
# args1.lamino_search_width          =  5.0,
# args1.lamino_search_step           =  0.25,
# args1.lamino_angle                 =  0,
# args1.lamino_start_row             =  0,
# args1.lamino_end_row               =  -1,
# args1.binning = 0
args1.minus_log =True

args2 = SimpleNamespace(
                       file_name                    = fname,
                       dark_file_name               = fname, 
                       flat_file_name               = fname, 
                       out_path_name                = '/data/tmpfdc/',
                       file_type                    = 'standard',
                       binning                      = 0,
                       blocked_views                = 'none',
                       remove_stripe_method         = 'vo',
                       rotation_axis                = -1,
                       center_search_width          =  50,
                       center_search_step           =  0.5,
                       nsino                        = '0.5',
                       nsino_per_chunk              = 8,
                       nproj_per_chunk              = 8,
                       start_row                    = 0,
                       end_row                      = -1,
                       start_column                 = 0,
                       end_column                   = -1,
                       start_proj                   = 0,
                       end_proj                     = -1,
                       rotation_axis_auto           = 'auto', 
                       rotation_axis_pairs          = '[0,0]',
                       rotation_axis_sift_threshold = 0.5,
                       rotation_axis_method         = 'sift',  
                       find_center_start_row        = 0,
                       find_center_end_row          = -1,
                       dtype                        = 'float16',

                       clear_folder                 = True,
                       fbp_filter                   = 'shepp',
                       dezinger                     = 0,

                       max_write_threads            = 8,
                       max_read_threads             = 4,
                       minus_log                    = True,
                       flat_linear                  = 'False',
    
    
    
    
    
    
    
    
    
    
                       reconstruction_type          = 'try',
                       reconstruction_algorithm     = 'fourierrec',
                       lamino_search_width          =  5.0,
                       lamino_search_step           =  0.25,
                       lamino_angle                 =  0,
                       lamino_start_row             =  0,
                       lamino_end_row               =  -1,
                       rotate_proj_angle            = 0,
                       rotate_proj_order            = 1,
                       retrieve_phase_method        = None,
                       pixel_size                   = meta_dict['measurement_instrument_detection_system_objective_resolution'][0] * 1e-4,
                       propagation_distance         = meta_dict['measurement_instrument_detector_motor_stack_setup_z'][0],
                       energy                       = meta_dict['measurement_instrument_monochromator_energy'][0],
                       retrieve_phase_alpha         = 0.2,
                      )

print('args******')
print(args)
print('args1*****')
print(args1)
print('args2*****')
print(args2)
print('**********')


args******
namespace(file_name='/data/2022-12/Luxi_173.h5', dark_file_name='/data/2022-12/Luxi_173.h5', flat_file_name='/data/2022-12/Luxi_173.h5', out_path_name='/data/tmpfdc/', file_type='standard', binning=0, blocked_views='none', remove_stripe_method='vo', rotation_axis=-1, center_search_width=50, center_search_step=0.5, nsino='0.5', nsino_per_chunk=8, nproj_per_chunk=8, start_row=0, end_row=-1, start_column=0, end_column=-1, start_proj=0, end_proj=-1, rotation_axis_auto='auto', rotation_axis_pairs='[0,0]', rotation_axis_sift_threshold=0.5, rotation_axis_method='sift', find_center_start_row=0, find_center_end_row=-1, dtype='float16', clear_folder=True, fbp_filter='shepp', dezinger=0, max_write_threads=8, max_read_threads=4, minus_log=True, flat_linear='False', reconstruction_type='try', reconstruction_algorithm='fourierrec', lamino_search_width=5.0, lamino_search_step=0.25, lamino_angle=0, lamino_start_row=0, lamino_end_row=-1, rotate_proj_angle=0, rotate_proj_order=1, retrieve_pha

In [ ]:
clrotthandle = tomocupy.FindCenter(args1)
args1.rotation_axis = clrotthandle.find_center()*2**args1.binning
print(f'set rotaion  axis {args1.rotation_axis}')

In [ ]:
clpthandle = tomocupy.GPURec(args)
clpthandle.recon_try()

In [14]:
center_search = dxchange.read_tiff_stack(args.out_path_name+ 'recon_slice0425_center1734.16.tiff', in)

TypeError: read_tiff_stack() missing 1 required positional argument: 'ind'